In [6]:
import pandas as pd
df = pd.read_csv('imdb.zip')

In [7]:
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [2]:
import joblib
tk = joblib.load('tokenizer.pkl')

In [3]:
seqs = tk.texts_to_sequences(df['review'])

In [8]:
seqs[0]

[4, 27, 27, 27, 287, 407, 1217, 13, 36, 4, 1218, 1219, 408, 142]

In [9]:
seqs[1]

[24, 522, 51, 12, 60, 409, 2, 736, 56, 46, 2, 337, 1220, 288, 5, 737, 738, 44]

In [4]:
import tensorflow as tf

In [5]:
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs)

In [13]:
import numpy as np

z = np.load('word_emb.npz')
e = z['emb']

In [15]:
emb2 = tf.keras.layers.Embedding(
    input_dim=tk.num_words + 1,
    output_dim=8,
    embeddings_initializer=tf.keras.initializers.Constant(e)
)

model = tf.keras.Sequential([
    emb2,
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 8)           16008     
_________________________________________________________________
global_average_pooling1d (Gl (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 16,089
Trainable params: 16,089
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [18]:
y = df['sentiment'].values

In [19]:
model.fit(pads, y)

32/32 [==============================] - 1s 5ms/step - loss: 0.7063 - accuracy: 0.4940
